In [1]:
import pandas as pd
import pytz
import pendulum

parameters = {
    'cities': ['Los_Angeles',
               'Chicago',
               'New_York',
               'London',
               'Berlin',
               'Moscow',
               'Dubai',
               'Shanghai',
               'Tokyo'],
    'cities_west_of_chicago': 1,
    'start_session': 17,
    'open_market': 9
}

timezone_cities = [ [tz for tz in pytz.all_timezones if city in tz][0] for city in parameters['cities']]
pendulum_for_each_city = [pendulum.today(tz) for tz in timezone_cities]

index_chicago = parameters['cities'].index('Chicago')
print("index_chicago==",index_chicago)
p_chicago = pendulum_for_each_city[index_chicago]

def get_offset_city(i):
    if i<parameters['cities_west_of_chicago']:
        return(-p_chicago.diff(pendulum_for_each_city[i]).in_hours())
    else:
        return(p_chicago.diff(pendulum_for_each_city[i]).in_hours())

def get_entry(h,o):
    return((parameters['start_session']+o+h)%24)

offset_cities = [get_offset_city(i) for i in range(len(pendulum_for_each_city))]
data = {_: [get_entry(_,o) for o in offset_cities] for _ in range(24)}
df = pd.DataFrame(data)
df.rename(index={_: parameters['cities'][_] for _ in range(len(parameters['cities']))}, inplace=True)

# Using DataFrame.insert() to add a column
index_open_market=int(df.loc[parameters['cities'][0],[df[col][0]==parameters['open_market'] for col in df.columns]].index[0])
df.insert(index_open_market, "|", ["|" for  city in parameters['cities']], True)

pd.options.display.max_columns = None
display(df)


index_chicago== 1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,|,18,19,20,21,22,23
Los_Angeles,15,16,17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,|,9,10,11,12,13,14
Chicago,17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,10,|,11,12,13,14,15,16
New_York,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,10,11,|,12,13,14,15,16,17
London,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,|,17,18,19,20,21,22
Berlin,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,|,18,19,20,21,22,23
Moscow,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,|,20,21,22,23,0,1
Dubai,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,|,21,22,23,0,1,2
Shanghai,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,|,1,2,3,4,5,6
Tokyo,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1,|,2,3,4,5,6,7
